## Загрузка данных

In [1]:
#Импортируем библиотеки
import pandas as pd
import numpy as np
import math

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
#Загружаем датасет
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
#Смотрим данные
print('-' * 100)
display(data.head(10))
print('-' * 100)
data.info()
print('-' * 100)
data.describe()

----------------------------------------------------------------------------------------------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
----------------------------------------------------------------------------------------------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


**Описание:**

В данных имеются 5 признаков: Пол, возраст, зарплата застрахованного, количество членов семьи.  
Целевой признак: количество страховых выплат клиенту за последние 5 лет.  

In [4]:
print(f'Количество дубликатов:', data.duplicated().sum())
print(f'Количество пропусков:', data.isna().sum().sum())

Количество дубликатов: 153
Количество пропусков: 0


In [5]:
# Подсчет уникальных значений для каждого признака
unique_counts = {
    'Пол': data['Пол'].nunique(),
    'Возраст': data['Возраст'].nunique(),
    'Зарплата': data['Зарплата'].nunique(),
    'Члены семьи': data['Члены семьи'].nunique(),
    'Страховые выплаты': data['Страховые выплаты'].nunique()
}

# Общее количество уникальных комбинаций
N = 1
for count in unique_counts.values():
    N *= count

# Количество записей
num_records = len(data)

# Вероятность отсутствия дубликатов
if N >= num_records:
    probability_no_duplicates = (math.factorial(N) / (math.factorial(N - num_records) * (N ** num_records)))
else:
    probability_no_duplicates = 0

print(f'Общее количество уникальных комбинаций: {N}')
print(f'Вероятность отсутствия дубликатов среди {num_records} записей: {probability_no_duplicates:.10f}')

Общее количество уникальных комбинаций: 2024736
Вероятность отсутствия дубликатов среди 5000 записей: 0.0020756182


**Выводы:**

С учетом полученных результатов, вероятность отсутствия дубликатов в наборе данных очень низка, что может говорить о том, что наличие дубликатов является обычным явлением и это признаки разных клиентов.

Пропуски в данных отсутствуют.

## Изменение качества линейной регрессии

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответьте на вопрос и обоснуйте решение.**

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)  

a. Изменится. Приведите примеры матриц.  
b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.  

**Ответ:** Качество линейной регрессии не изменится, если признаки умножаются на обратимую матрицу.

**Обоснование:** Это обосновывается следующим образом:

Исходно у нас есть линейная регрессия, моделирующая зависимость между признаками \(X\) и целевым признаком \(y\):

$y = Xw$

Где:
- $y$- вектор целевых значений,
- $X$ - матрица признаков,
- $w$ - вектор весов (параметров) модели.

Теперь мы умножим признаки $X$ на обратимую матрицу $A$ и получим новую матрицу признаков $X'$:

$X' = XA$

Затем обучим новую линейную регрессию на $X'$ и получим новый вектор весов $w'$:

$y = X'w'$

Из уравнения $y = Xw$ и $X' = XA$, мы можем выразить $w'$ как:

$w' = A^{-1}w$

Это означает, что новые веса модели $w'$ связаны с исходными весами $w$ умножением на обратную матрицу $A^{-1}$. Важно отметить, что $A^{-1}$ - это обратимая матрица, поэтому никакая информация не теряется в процессе преобразования.

Таким образом, хотя признаки были изменены (умножены на матрицу $A$), модель линейной регрессии на новых признаках будет иметь ту же функциональную форму и качество, что и исходная модель. Качество линейной регрессии не изменится.

**Алгоритм преобразования данных для защиты информации клиентов в задаче линейной регрессии:**

1. **Генерация обратимой матрицы $A$**:
   - Для обеспечения безопасности данных, генерируется случайная обратимая матрица $A$ размером $n x n$, где $n$ - количество признаков.

2. **Преобразование данных**:
   - Признаки данных умножаются на матрицу $A$, получая новую матрицу признаков $X'$.

3. **Обучение линейной регрессии**:
   - Модель линейной регрессии обучается на новой матрице признаков $X'$, с использованием библиотеки scikit-learn.

4. **Оценка качества модели**:
   - Эффективность преобразования оценивается путем измерения качества модели на тестовых данных с использованием метрик, таких как $R^2$ score или других метрик регрессии, чтобы убедиться, что качество модели остается на приемлемом уровне.

**Обоснование:**

Линейная регрессия является линейной моделью, и она линейно зависит от признаков. Умножение признаков на обратимую матрицу 
$A$ также представляет собой линейное преобразование. Поэтому линейная регрессия будет работать с преобразованными данными так же, как и с исходными данными, так как линейная модель сохраняет линейные зависимости.

Преобразование данных путем умножения на обратимую матрицу не разрушает информацию о линейных зависимостях между признаками, и эта информация может быть восстановлена после обратного преобразования данных.

Матрица $A$ обратима, что гарантирует возможность восстановления исходных данных из преобразованных данных при необходимости. Это важно для целостности исходной информации.

## Проверка алгоритма

In [6]:
# Выделим признаки и целевой признак
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

In [14]:
# Шаг 1: Создание обратимой матрицы A
n_features = features.shape[1]
A = np.random.rand(n_features, n_features)

# Проверка на обратимость и генерация новой матрицы
while np.linalg.det(A) == 0:
    A = np.random.rand(n_features, n_features)

# Шаг 2: Преобразование признаков
features_transformed = np.dot(features, A)

# Шаг 3: Обучение модели линейной регрессии на исходных признаках
model_original = LinearRegression()
model_original.fit(features, target)
predictions_original = model_original.predict(features)
r2_before = r2_score(target, predictions_original)

# Обучение модели линейной регрессии на преобразованных признаках
model_transformed = LinearRegression()
model_transformed.fit(features_transformed, target)
predictions_transformed = model_transformed.predict(features_transformed)
r2_after = r2_score(target, predictions_transformed)

# Шаг 4: Вывод результатов
print(f"R2 score до преобразования данных: {r2_before}")
print(f"R2 score после преобразования данных: {r2_after}")

R2 score до преобразования данных: 0.4249455028666801
R2 score после преобразования данных: 0.42494550286670385


**Вывод:**

R2 score до и после преобразования данных практически одинаковы, что подтверждает, что качество линейной регрессии не изменилось после преобразования данных. Это подтверждает корректность примененного алгоритма для защиты данных клиентов в задаче линейной регрессии.